# Siege The Castle Model 2

**Rules/Assumptions**
- 3 player game
- 25% percent chance between 4 move options
    - *if 150 or less life, bolster*
    - 50% for attack options
        - *if opponent has less than 1000 gold, don't attack money*
- no bolster charge
- no tax is always worth 3750 gold (2500 + 625*2)
- default settings

In [11]:
NumberOfSimulatedGames = 10000

In [12]:
import random
import pandas as pd

# Read in turn outcomes and put into dictionary

import csv
reader = csv.reader(open('C:\\Users\\brian\\Documents\\GitHub\\SiegeTheCastle_Sim\\01_load\\load.csv'))

turnOutcomes = {}
for row in reader:
    key = row[0]
    if key in turnOutcomes:
        # implement your duplicate row handling here
        pass
    turnOutcomes[key] = row[1:]

# Create DataFrames
dfTurns = pd.DataFrame(columns=['Game', 'Turn', 'Actions'])
dfGames = pd.DataFrame(columns=['Game','NumTurns','EndKingLife','EndKingGold','EndClan1Life','EndClan2Life'])

# Set Initial Values / Settings

KingStartingLives = 625
KingStartingGold = 20000
Clan1StartingLives = 400
Clan1StartingGold = 0
Clan2StartingLives = 400
Clan2StartingGold = 0
#print(KingStartingLives, KingStartingGold, Clan1StartingLives, Clan1StartingGold, Clan2StartingLives, Clan2StartingGold)

##### START GAME LOOPS
GameCount = 1

while GameCount <= NumberOfSimulatedGames:

    # Set Load Current Values at start of round
    KingCurrentLives = KingStartingLives
    KingCurrentGold = KingStartingGold
    Clan1CurrentLives = Clan1StartingLives
    Clan1CurrentGold = Clan1StartingGold
    Clan2CurrentLives = Clan2StartingLives
    Clan2CurrentGold = Clan2StartingGold
    #print(KingCurrentLives, KingCurrentGold, Clan1CurrentLives, Clan1CurrentGold, Clan2CurrentLives, Clan2CurrentGold)

    # Set End Game Condition Flags
    EndKingLife = 0
    EndKingGold = 0
    EndClan1Life = 0
    EndClan2Life = 0
    #print(EndKingLife,EndKingGold,EndClan1Life, EndClan2Life)

    ### START TURN LOOP
    TurnCount = 1

    while EndKingLife+EndKingGold+EndClan1Life+EndClan2Life == 0:
        #print(TurnCount)
        #### Generate a Turn Action

        KingAction = random.randint(1,4)
        
        ### 11/28/2020: IMPROVE TURN LOGIC ###
        ### - If a King Attack or failed attack will cause you to lose, bolster
        ### - If opposing clan has no gold, don't attack them for gold
        
        if Clan1CurrentLives <= 150:
            Clan1Action = 4
        else:
            Clan1Action = random.randint(1,4)
        
        if Clan2CurrentLives <= 150:
            Clan2Action = 4
        else:
            Clan2Action = random.randint(1,4)

        if KingAction == 3:
            KingAttack = random.randint(1,2)
        else:
            KingAttack = 0

        if Clan1Action == 3:
            if Clan2CurrentGold <= 1000:
                Clan1Attack = 2
            else:
                Clan1Attack = random.randint(1,2)
        else:
            Clan1Attack = 0

        if Clan2Action == 3:
            if Clan1CurrentGold <= 1000:
                Clan2Attack = 2
            else:
                Clan2Attack = random.randint(1,2)
        else:
            Clan2Attack = 0

        #print(KingAction, Clan1Action, Clan2Action)
        #print(KingAttack, Clan1Attack, Clan2Attack)

        # Map Turn Action into turn key

        if KingAction == 1:
            KingMove = 'F'
        if KingAction == 2:
            KingMove = 'P'
        if KingAction == 3:
            if KingAttack == 1:
                KingMove = '1'
            else:
                KingMove = '2'
        if KingAction == 4:
            KingMove = 'T'
            
        if Clan1Action == 1:
            Clan1Move = 'S'
        if Clan1Action == 2:
            Clan1Move = 'P'
        if Clan1Action == 3:
            if Clan1Attack == 1:
                Clan1Move = 'M'
            else:
                Clan1Move = 'L'
        if Clan1Action == 4:
            Clan1Move = 'B'

        if Clan2Action == 1:
            Clan2Move = 'S'
        if Clan2Action == 2:
            Clan2Move = 'P'
        if Clan2Action == 3:
            if Clan2Attack == 1:
                Clan2Move = 'M'
            else:
                Clan2Move = 'L'
        if Clan2Action == 4:
            Clan2Move = 'B'

        #print(KingMove, Clan1Move, Clan2Move)

        turnKey = KingMove+Clan1Move+Clan2Move

        #print(turnKey)

        # pull turn outcomes from dictionary using turnKey

        moves = turnOutcomes[turnKey]

        #print(moves)

        KingLivesChange = int(moves[3])
        KingGoldChange = int(moves[4])
        Clan1LivesChange = int(moves[5])
        Clan1GoldChange = int(moves[6])
        Clan2LivesChange = int(moves[7])
        Clan2GoldChange = int(moves[8])

        #print(KingLivesChange, KingGoldChange, Clan1LivesChange, Clan1GoldChange, Clan2LivesChange, Clan2GoldChange)

        # Apply Turn Outcome
        KingCurrentLives += KingLivesChange
        KingCurrentGold += KingGoldChange
        Clan1CurrentLives += Clan1LivesChange
        Clan1CurrentGold += Clan1GoldChange
        Clan2CurrentLives += Clan2LivesChange
        Clan2CurrentGold += Clan2GoldChange
        #print(KingCurrentLives, KingCurrentGold, Clan1CurrentLives, Clan1CurrentGold, Clan2CurrentLives, Clan2CurrentGold)

        if KingCurrentLives <= 0:
            EndKingLife = 1
        if KingCurrentGold <= 0:
            EndKingGold = 1
        if Clan1CurrentLives <= 0:
            EndClan1Life = 1
        if Clan2CurrentLives <= 0:
            EndClan2Life = 1

        dfTurnsTemp = pd.DataFrame([[GameCount,TurnCount,turnKey]],columns=['Game', 'Turn', 'Actions'])
        dfTurns = dfTurns.append(dfTurnsTemp)
        # increment Turn Count
        TurnCount += 1

        #print(KingCurrentLives, KingCurrentGold, Clan1CurrentLives, Clan1CurrentGold, Clan2CurrentLives, Clan2CurrentGold)

####### TURN LOOP END

    dfGamesTemp = pd.DataFrame([[GameCount,TurnCount,EndKingLife,EndKingGold,EndClan1Life,EndClan2Life]], columns=['Game','NumTurns','EndKingLife','EndKingGold','EndClan1Life','EndClan2Life'])
    dfGames = dfGames.append(dfGamesTemp)
    
    GameCount += 1

###### GAME LOOP END


In [13]:
print('Count of Games: '+str(dfGames['Game'].max()))
print('Average Turns: '+ str(dfGames['NumTurns'].mean()))
print('Games Ended by King Running Out of Life: '+ str(dfGames['EndKingLife'].sum()))
print('Games Ended by King Running Out of Gold: '+ str(dfGames['EndKingGold'].sum()))
print('Games Ended by Clan 1 Running out of Life: '+ str(dfGames['EndClan1Life'].sum()))
print('Games Ended by Clan 2 Running out of Life: '+ str(dfGames['EndClan2Life'].sum()))

Count of Games: 10000
Average Turns: 19.606
Games Ended by King Running Out of Life: 2698
Games Ended by King Running Out of Gold: 202
Games Ended by Clan 1 Running out of Life: 4059
Games Ended by Clan 2 Running out of Life: 3192


In [6]:
dfTurns.head()

,Game,Turn,Actions
0,1,1,TPS
0,1,2,FSL
0,1,3,TPL
0,1,4,PLP
0,1,5,PBS
